In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

Reading data from the SPARCS dataset into a dataframe

In [2]:
data = pd.read_csv("../../datasets/electronic_health_record/input/electronic_health_record_dataset.csv", low_memory=False)

In [3]:
health_data = data.copy()

In [4]:
health_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2061634 entries, 0 to 2061633
Data columns (total 33 columns):
 #   Column                               Dtype  
---  ------                               -----  
 0   Hospital Service Area                object 
 1   Hospital County                      object 
 2   Operating Certificate Number         float64
 3   Permanent Facility Id                float64
 4   Facility Name                        object 
 5   Age Group                            object 
 6   Zip Code - 3 digits                  object 
 7   Gender                               object 
 8   Race                                 object 
 9   Ethnicity                            object 
 10  Length of Stay                       object 
 11  Type of Admission                    object 
 12  Patient Disposition                  object 
 13  Discharge Year                       int64  
 14  CCSR Diagnosis Code                  object 
 15  CCSR Diagnosis Description      

Attributes and unique value present in them

In [5]:
def print_unique_values(dataframe):
    for col in dataframe.columns:
        print(f"{col}:\n{dataframe[col].unique()}", end="\n\n")

In [6]:
print_unique_values(health_data)

Hospital Service Area:
['New York City' 'Hudson Valley' nan 'Long Island' 'Capital/Adirond'
 'Central NY' 'Finger Lakes' 'Western NY' 'Southern Tier']

Hospital County:
['Bronx' 'Rockland' nan 'Manhattan' 'Westchester' 'Kings' 'Queens'
 'Orange' 'Nassau' 'Sullivan' 'Otsego' 'Herkimer' 'Delaware' 'Monroe'
 'Ontario' 'Cortland' 'Columbia' 'Albany' 'Suffolk' 'Onondaga' 'Madison'
 'Steuben' 'Cayuga' 'Montgomery' 'Erie' 'Jefferson' 'Oswego' 'Yates'
 'Wayne' 'Genesee' 'Schoharie' 'Ulster' 'Oneida' 'Schenectady' 'Broome'
 'St Lawrence' 'Schuyler' 'Richmond' 'Niagara' 'Chemung' 'Essex'
 'Chautauqua' 'Dutchess' 'Putnam' 'Chenango' 'Tompkins' 'Warren' 'Fulton'
 'Wyoming' 'Franklin' 'Cattaraugus' 'Saratoga' 'Lewis' 'Livingston'
 'Allegany' 'Orleans' 'Clinton']

Operating Certificate Number:
[7000006. 4324000.      nan 7002054. 7002002. 7002032. 5902001. 7001016.
 7002001. 7000024. 7002024. 7000002. 7001009. 7002050. 7000008. 7002021.
 7003007. 7003000. 7002009. 7001045. 5903001. 5904001. 3522000.

Checking for exact duplicate records and removing them

In [7]:
print(f"Duplicate records: {health_data.duplicated().sum()}")
health_data = health_data.drop_duplicates()

Duplicate records: 3844


Treating missing values

Dropping columns having more than 50% missing values 

In [8]:
missing_values_percentage = (health_data.isnull().mean() * 100)
print(missing_values_percentage.sort_values(ascending=False))

columns_to_drop = missing_values_percentage[missing_values_percentage > 50].index

health_data = health_data.drop(columns_to_drop, axis=1)

Birth Weight                           90.185636
Payment Typology 3                     86.008679
Payment Typology 2                     52.958757
CCSR Procedure Code                    27.484243
CCSR Procedure Description             27.484243
Zip Code - 3 digits                     1.957100
Operating Certificate Number            0.282876
Hospital County                         0.255128
Hospital Service Area                   0.255128
Permanent Facility Id                   0.255128
Total Costs                             0.206435
APR Risk of Mortality                   0.030761
APR Severity of Illness Description     0.030761
Length of Stay                          0.000000
Total Charges                           0.000000
Emergency Department Indicator          0.000000
Facility Name                           0.000000
Age Group                               0.000000
Payment Typology 1                      0.000000
APR Medical Surgical Description        0.000000
APR Severity of Illn

Dropping ['Operating Certificate Number','Discharge Year'] since they don't have any significant importance in the prediction and analysis

In [9]:
health_data = health_data.drop(['Operating Certificate Number','Discharge Year'], axis = 1)

Imputing missing values in 
- CCSR Procedure Code on the basis of mode of its grouped CCSR Diagnosis Code
- CCSR Procedure Description on the basis of mode of its grouped CCSR Procedure Code

In [10]:
def map_and_fill_missing_values(dataframe, from_col, to_col):
   temp_df = dataframe.copy()

   map = temp_df.groupby(from_col)[to_col].agg(
      lambda x: x.value_counts().idxmax() if x.count() > 0 else None
   ).to_dict()
    
   temp_df[to_col] = temp_df[to_col].fillna(temp_df[from_col].map(map))
   
   return temp_df

In [11]:
health_data = map_and_fill_missing_values(health_data, "CCSR Diagnosis Code", "CCSR Procedure Code")

In [12]:
health_data = map_and_fill_missing_values(health_data, "CCSR Procedure Code", "CCSR Procedure Description")

Checking for missing values

In [13]:
health_data.isna().sum()

Hospital Service Area                   5250
Hospital County                         5250
Permanent Facility Id                   5250
Facility Name                              0
Age Group                                  0
Zip Code - 3 digits                    40273
Gender                                     0
Race                                       0
Ethnicity                                  0
Length of Stay                             0
Type of Admission                          0
Patient Disposition                        0
CCSR Diagnosis Code                        0
CCSR Diagnosis Description                 0
CCSR Procedure Code                        2
CCSR Procedure Description                 2
APR DRG Code                               0
APR DRG Description                        0
APR MDC Code                               0
APR MDC Description                        0
APR Severity of Illness Code               0
APR Severity of Illness Description      633
APR Risk o

Dropping records having missing values since the count is negligible

In [14]:
health_data = health_data.dropna(axis=0)

health_data.isna().sum()

Hospital Service Area                  0
Hospital County                        0
Permanent Facility Id                  0
Facility Name                          0
Age Group                              0
Zip Code - 3 digits                    0
Gender                                 0
Race                                   0
Ethnicity                              0
Length of Stay                         0
Type of Admission                      0
Patient Disposition                    0
CCSR Diagnosis Code                    0
CCSR Diagnosis Description             0
CCSR Procedure Code                    0
CCSR Procedure Description             0
APR DRG Code                           0
APR DRG Description                    0
APR MDC Code                           0
APR MDC Description                    0
APR Severity of Illness Code           0
APR Severity of Illness Description    0
APR Risk of Mortality                  0
APR Medical Surgical Description       0
Payment Typology

Imputing missing/unknown values of ["Gender", "Type of Admission"] columns with corresponding mode value

In [15]:
def impute_missing_values_with_mode(dataframe, column, missing_value=None):
    temp_df = dataframe.copy()
    mode_value = temp_df[column].mode()[0]
    temp_df[column] = temp_df[column].replace(missing_value, mode_value)
    return temp_df

In [16]:
health_data = impute_missing_values_with_mode(health_data, "Gender", missing_value="U")

In [17]:
health_data = impute_missing_values_with_mode(health_data, "Type of Admission", missing_value="Not Available")

Treating unknown ethinicity using race

In [18]:
mode_race_for_unknown_ethinicity = health_data[health_data["Ethnicity"] == "Unknown"]["Race"].mode()[0]
mode_ethinicity_for_selection = health_data[health_data["Race"] == mode_race_for_unknown_ethinicity]["Ethnicity"].mode()[0]
health_data["Ethnicity"] = health_data["Ethnicity"].replace("Unknown", mode_ethinicity_for_selection)

Converting to different data types:
- Limiting "120 +" stay duration to 120 and converting column to integer type
- Replacing "OOS" zip code with largest 3 digit integer and converting values to int type
- Converting Permanent Facility Id to type int

In [19]:
health_data["Length of Stay"] = health_data["Length of Stay"].replace("120 +", "120").astype(int)

In [20]:
health_data["Zip Code - 3 digits"] = health_data["Zip Code - 3 digits"].replace("OOS", "999").astype(int)

In [21]:
health_data["Permanent Facility Id"] = health_data["Permanent Facility Id"].astype(int)

In [22]:
health_data["Total Charges"] = health_data["Total Charges"].replace(",", "", regex=True).astype(float)

In [23]:
health_data["Total Costs"] = health_data["Total Costs"].replace(",", "", regex=True).astype(float)

In [24]:
health_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2012747 entries, 0 to 2061633
Data columns (total 28 columns):
 #   Column                               Dtype  
---  ------                               -----  
 0   Hospital Service Area                object 
 1   Hospital County                      object 
 2   Permanent Facility Id                int32  
 3   Facility Name                        object 
 4   Age Group                            object 
 5   Zip Code - 3 digits                  int32  
 6   Gender                               object 
 7   Race                                 object 
 8   Ethnicity                            object 
 9   Length of Stay                       int32  
 10  Type of Admission                    object 
 11  Patient Disposition                  object 
 12  CCSR Diagnosis Code                  object 
 13  CCSR Diagnosis Description           object 
 14  CCSR Procedure Code                  object 
 15  CCSR Procedure Description           

Dealing with Outliers

In [25]:
def detect_and_impute_outliers(dataframe, column):
    temp_df = dataframe.copy()

    q1 = dataframe[column].quantile(0.25)
    q3 = dataframe[column].quantile(0.75)
    iqr = q3 - q1

    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr

    outlier_count = temp_df[(temp_df[column] < lower_bound) | (temp_df[column] > upper_bound)].shape[0]
    print(f"Number of outliers in {column}: {outlier_count}")

    temp_df[column] = temp_df[column].mask(temp_df[column] < lower_bound, lower_bound, axis=0)
    temp_df[column] = temp_df[column].mask(temp_df[column] > upper_bound, upper_bound, axis=0)
    return temp_df

In [26]:
health_data = detect_and_impute_outliers(health_data, "Total Charges")

Number of outliers in Total Charges: 180447


In [27]:
health_data = detect_and_impute_outliers(health_data, "Total Costs")

Number of outliers in Total Costs: 176491


In [28]:
health_data.head()

,Hospital Service Area,Hospital County,Permanent Facility Id,Facility Name,Age Group,Zip Code - 3 digits,Gender,Race,Ethnicity,Length of Stay,...,APR MDC Code,APR MDC Description,APR Severity of Illness Code,APR Severity of Illness Description,APR Risk of Mortality,APR Medical Surgical Description,Payment Typology 1,Emergency Department Indicator,Total Charges,Total Costs
0,New York City,Bronx,3058,Montefiore Med Center - Jack D Weiler Hosp of ...,50 to 69,107,F,White,Not Span/Hispanic,3,...,4,DISEASES AND DISORDERS OF THE RESPIRATORY SYSTEM,3,Major,Major,Medical,Medicaid,Y,51514.62,7552.54
1,New York City,Bronx,3058,Montefiore Med Center - Jack D Weiler Hosp of ...,18 to 29,104,M,Black/African American,Spanish/Hispanic,1,...,9,"DISEASES AND DISORDERS OF THE SKIN, SUBCUTANEO...",2,Moderate,Minor,Medical,Medicaid,Y,25370.86,3469.55
2,New York City,Bronx,3058,Montefiore Med Center - Jack D Weiler Hosp of ...,18 to 29,104,F,Other Race,Spanish/Hispanic,2,...,14,"PREGNANCY, CHILDBIRTH AND THE PUERPERIUM",1,Minor,Minor,Medical,Medicaid,N,23876.78,6180.33
3,New York City,Bronx,3058,Montefiore Med Center - Jack D Weiler Hosp of ...,18 to 29,100,F,Black/African American,Not Span/Hispanic,2,...,14,"PREGNANCY, CHILDBIRTH AND THE PUERPERIUM",2,Moderate,Minor,Medical,Medicaid,Y,43319.05,12588.93
4,New York City,Bronx,1168,Montefiore Medical Center-Wakefield Hospital,18 to 29,104,M,Other Race,Spanish/Hispanic,4,...,19,MENTAL DISEASES AND DISORDERS,2,Moderate,Moderate,Medical,Medicaid,Y,40266.23,10355.99


Dropping redundant columns

In [29]:
for col in health_data.columns:
    print(f"{col}: {health_data[col].nunique()}")

Hospital Service Area: 8
Hospital County: 56
Permanent Facility Id: 200
Facility Name: 199
Age Group: 5
Zip Code - 3 digits: 50
Gender: 2
Race: 4
Ethnicity: 3
Length of Stay: 120
Type of Admission: 5
Patient Disposition: 19
CCSR Diagnosis Code: 472
CCSR Diagnosis Description: 472
CCSR Procedure Code: 319
CCSR Procedure Description: 319
APR DRG Code: 326
APR DRG Description: 326
APR MDC Code: 24
APR MDC Description: 24
APR Severity of Illness Code: 4
APR Severity of Illness Description: 4
APR Risk of Mortality: 4
APR Medical Surgical Description: 2
Payment Typology 1: 9
Emergency Department Indicator: 2
Total Charges: 1589990
Total Costs: 1355856


In [30]:
redundant_columns = ["Facility Name", "CCSR Diagnosis Description", "CCSR Procedure Description", "APR DRG Description", "APR MDC Description", "APR Severity of Illness Description"]
health_data = health_data.drop(redundant_columns, axis=1)

In [31]:
for col in health_data.columns:
    if health_data[col].dtype == "object":
        print(f"{col}: {health_data[col].nunique()}")

Hospital Service Area: 8
Hospital County: 56
Age Group: 5
Gender: 2
Race: 4
Ethnicity: 3
Type of Admission: 5
Patient Disposition: 19
CCSR Diagnosis Code: 472
CCSR Procedure Code: 319
APR Risk of Mortality: 4
APR Medical Surgical Description: 2
Payment Typology 1: 9
Emergency Department Indicator: 2


Encoding categorical data:
- Target Encoding: ["Hospital Service Area", "Hospital County", "Patient Disposition", "CCSR Diagnosis Code", "CCSR Procedure Code"]
- Label Encoding: ['Age Group']
- Direct mapping: ['APR Risk of Mortality', 'Payment Typology 1']
- One Hot Encoding: ["Gender", "Race", "Ethnicity", "Type of Admission", "APR Medical Surgical Description", "Emergency Department Indicator"]

In [32]:
target_encoding_columns = ["Hospital Service Area", "Hospital County", "Patient Disposition", "CCSR Diagnosis Code", "CCSR Procedure Code"]
for col in target_encoding_columns:
    health_data[col] = health_data[col].map(dict(health_data.groupby(col)["Length of Stay"].mean()))

In [33]:
health_data['Age Group'] = LabelEncoder().fit_transform(health_data['Age Group'])

In [34]:
health_data['APR Risk of Mortality'] = health_data['APR Risk of Mortality'].map({'Minor': 0, 'Moderate': 1, 'Major': 2, 'Extreme': 3})

In [35]:
temp_dict = dict(health_data['Payment Typology 1'].value_counts() / len(health_data))
health_data['Payment Typology 1'] = health_data['Payment Typology 1'].map(temp_dict)

In [36]:
for col in health_data.columns:
    if health_data[col].dtype == "object":
        print(f"{col}: {health_data[col].nunique()}")

Gender: 2
Race: 4
Ethnicity: 3
Type of Admission: 5
APR Medical Surgical Description: 2
Emergency Department Indicator: 2


In [42]:
def one_hot_encoding(dataframe, columns):
    encoded_df = pd.get_dummies(dataframe, columns=columns)
    return encoded_df

In [43]:
one_hot_encoding_columns = ["Gender", "Race", "Ethnicity", "Type of Admission", "APR Medical Surgical Description", "Emergency Department Indicator"]

health_data = one_hot_encoding(health_data, one_hot_encoding_columns)

In [45]:
health_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2012747 entries, 0 to 2061633
Data columns (total 34 columns):
 #   Column                                     Dtype  
---  ------                                     -----  
 0   Hospital Service Area                      float64
 1   Hospital County                            float64
 2   Permanent Facility Id                      int32  
 3   Age Group                                  int32  
 4   Zip Code - 3 digits                        int32  
 5   Length of Stay                             int32  
 6   Patient Disposition                        float64
 7   CCSR Diagnosis Code                        float64
 8   CCSR Procedure Code                        float64
 9   APR DRG Code                               int64  
 10  APR MDC Code                               int64  
 11  APR Severity of Illness Code               int64  
 12  APR Risk of Mortality                      int64  
 13  Payment Typology 1                         floa

In [ ]:
health_data.to_csv('../../datasets/electronic-health-record/output/transformed_electronic_health_record_dataset.csv', index=False)